In [118]:
from pyspark.sql import SparkSession

In [162]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("restaurant-review-average") \
    .master("local[*]") \
    .getOrCreate()

In [163]:
spark = SparkSession.builder.appName("HeaderRemoval").master("local[*]").getOrCreate()
sc = spark.sparkContext

In [164]:
sc = spark.sparkContext  # 필요 시 RDD API를 위해 사용

In [165]:
file_path = "file:///home/jovyan/work/start_spark/start_spark/running_spark_data/restaurant_reviews.csv"
lines = sc.textFile(file_path)

In [166]:
lines = sc.textFile("file:///home/jovyan/work/start_spark/start_spark/running_spark_data/restaurant_reviews.csv")

lines.take(5)

['id,item,cateogry,reviews,',
 '0,짜장면,중식,125,',
 '1,짬뽕,중식,235,',
 '2,김밥,분식,32,',
 '3,떡볶이,분식,534,']

In [167]:
data = [
    (0, "짜장면", "중식", 125),
    (1, "짬뽕", "중식", 235),
    (2, "김밥", "분식", 32),
    (3, "떡볶이", "분식", 534),
    (4, "라멘", "일식", 223),
    (5, "돈가스", "일식", 52),
    (6, "우동", "일식", 12),
    (7, "쌀국수", "아시안", 312),
    (8, "햄버거", "패스트푸드", 12),
    (9, "치킨", "패스트푸드", 23),
]

In [168]:

header = lines.first()
print(f"헤더: {header}")

# 2. filter 변환을 사용하여 헤더를 제외한 데이터만 남깁니다.
data_without_header = lines.filter(lambda line: line != header)

# 헤더가 제거된 RDD의 첫 5개 항목을 확인합니다.
print("\n--- 헤더가 제거된 데이터 미리보기 (상위 5개) ---")
for line in data_without_header.take(5):
    print(line)


헤더: id,item,cateogry,reviews,

--- 헤더가 제거된 데이터 미리보기 (상위 5개) ---
0,짜장면,중식,125,
1,짬뽕,중식,235,
2,김밥,분식,32,
3,떡볶이,분식,534,
4,라멘,일식,223,


In [ ]:
#총건수확인 첫줄 데이터 분할, map으로 split

In [169]:

data_rdd = spark.sparkContext.parallelize(data)

# RDD의 총 건수를 계산합니다.
total_count_rdd = data_rdd.count()

# 결과를 출력합니다.
print(f"RDD의 총 건수는 {total_count_rdd}개입니다.")


RDD의 총 건수는 10개입니다.


In [170]:
def parse(row):
    fields = row.split(",")
    
    category = fields[2] #카테고리
    
    # reviews는 정수로 parse
    reviews = fields[3] # 리뷰수
    reviews = int(reviews)
    
    return category, reviews

In [171]:
parse('0,짜장면,중식,125,')

('중식', 125)

In [172]:
# 각 라인을 쉼표(,) 기준으로 분할하여 리스트 형태로 만듭니다
filtered_lines= data_without_header.map(lambda line: line.split(','))

In [173]:
category_reviews = filtered_lines.map(parse)
category_reviews

PythonRDD[9] at RDD at PythonRDD.scala:53

In [176]:
filtered_lines.collect()

[['0', '짜장면', '중식', '125', ''],
 ['1', '짬뽕', '중식', '235', ''],
 ['2', '김밥', '분식', '32', ''],
 ['3', '떡볶이', '분식', '534', ''],
 ['4', '라멘', '일식', '223', ''],
 ['5', '돈가스', '일식', '52', ''],
 ['6', '우동', '일식', '12', ''],
 ['7', '쌀국수', '아시안', '312', ''],
 ['8', '햄버거', '패스트푸드', '12', ''],
 ['9', '치킨', '패스트푸드', '23']]

In [161]:
spark.stop()

In [178]:
parse('0,짜장면,중식,125,')

('중식', 125)

In [191]:
categoryReviews = filtered_lines.map(parse).persist() # categoryReviews RDD는 하나만 존재하는 RDD
categoryReviews

PythonRDD[21] at RDD at PythonRDD.scala:53

In [193]:
#narrow Transformations
#1:1변환 -> 하나의 열을 다룰 때 다른 데이터가 필요없는 경우
#filter(), map(), flatmap(), sample(), union()


In [200]:
#flat map()

In [194]:
rdd = sc.parallelize([1, 2, 3])
rdd_map = rdd.map(lambda x: [x, x + 1])  # => [[1, 2], [2, 3], [3, 4]]
rdd_map.collect()

[[1, 2], [2, 3], [3, 4]]

In [195]:
rdd_flatmap = rdd.flatMap(lambda x: [x, x + 1])  # => [1, 2, 2, 3, 3, 4]
rdd_flatmap.collect()

[1, 2, 2, 3, 3, 4]

In [199]:
movies = [
    "그린 북",
    "매트릭스",
    "토이 스토리",
    "캐스트 어웨이",
    "포드 V 페라리",
    "보헤미안 랩소디",
    "빽 투 더 퓨처",
    "반지의 제왕",
    "죽은 시인의 사회"
]

In [202]:
moviesRDD = sc.parallelize(movies)
moviesRDD

ParallelCollectionRDD[50] at readRDDFromFile at PythonRDD.scala:289

In [203]:
#job 추가 x
flatMovies = moviesRDD.flatMap(lambda x : x.split(" "))
flatMovies.collect()

['그린',
 '북',
 '매트릭스',
 '토이',
 '스토리',
 '캐스트',
 '어웨이',
 '포드',
 'V',
 '페라리',
 '보헤미안',
 '랩소디',
 '빽',
 '투',
 '더',
 '퓨처',
 '반지의',
 '제왕',
 '죽은',
 '시인의',
 '사회']

In [205]:
# 집합 Transformation
num1 = sc.parallelize([1, 2, 3, 4, 5])
num2 = sc.parallelize([4, 5, 6, 7, 8, 9, 10])


In [208]:
num1.intersection(num2).collect()
# 합집합 구하기 - union
num_union = num1.union(num2)
num_union
num_union.collect()

[1, 2, 3, 4, 5, 4, 5, 6, 7, 8, 9, 10]

In [217]:
# 차집합 구하기 - subtract
num1.subtract(num2).collect()

[1, 2, 3]

In [220]:
# 데이터 랜덤 추출 - sample(withReplacement, fraction, seed=None)
# withReplacement : True -> 중복 추출


In [231]:
# withReplacement : True -> 중복 추출
num_union.sample(True, 0.3).collect()

[9]

In [232]:
foods = sc.parallelize([
    "짜장면", "마라탕", "짬뽕", "떡볶이", "쌀국수", "짬뽕", "짜장면", "짜장면", "짜장면", "라면", "우동", "라면"
])
foods

ParallelCollectionRDD[143] at readRDDFromFile at PythonRDD.scala:289

In [247]:
# 그룹핑의 기준을 문자열의 첫 번째 글자로 설정
foodsGroup = foods.groupBy(lambda x : x[0])
foodsGroup

PythonRDD[169] at RDD at PythonRDD.scala:53

In [243]:
res=foodsGroup.collect()

In [241]:
res

[('짜', <pyspark.resultiterable.ResultIterable at 0x7ff71bc9f850>),
 ('짬', <pyspark.resultiterable.ResultIterable at 0x7ff71bc9d310>),
 ('쌀', <pyspark.resultiterable.ResultIterable at 0x7ff71bc9d990>),
 ('라', <pyspark.resultiterable.ResultIterable at 0x7ff71bc9ce90>),
 ('우', <pyspark.resultiterable.ResultIterable at 0x7ff71bc9cf50>),
 ('마', <pyspark.resultiterable.ResultIterable at 0x7ff71bc9ffd0>),
 ('떡', <pyspark.resultiterable.ResultIterable at 0x7ff71bc9e390>)]

In [244]:
for (k, v) in res:
    print(k, list(v))

짜 ['짜장면', '짜장면', '짜장면', '짜장면']
짬 ['짬뽕', '짬뽕']
쌀 ['쌀국수']
라 ['라면', '라면']
우 ['우동']
마 ['마라탕']
떡 ['떡볶이']


In [235]:
# 내로우 narrow  트랜스포메이션

In [249]:
### 2. Narrow 트랜스포메이션 예제
numbers = sc.parallelize([1, 2, 3, 4, 5])
numbers.collect()
# 1:1 변환 (Narrow Transformation)

[1, 2, 3, 4, 5]

In [257]:
union=numbers.union(sc.parallelize([6,7,8]))
union

UnionRDD[189] at union at <unknown>:0

In [258]:
kv_rdd = sc.parallelize([("apple",1), ("banana",2) , ("cherry",3) ])

In [260]:
mapp_values = kv_rdd.mapValues(lambda x: x*10)
mapp_values.collect()

[('apple', 10), ('banana', 20), ('cherry', 30)]

In [263]:
grouped_rdd = kv_rdd.groupByKey()
grouped_rdd.collect()

[('apple', <pyspark.resultiterable.ResultIterable at 0x7ff71a77c490>),
 ('banana', <pyspark.resultiterable.ResultIterable at 0x7ff71a60f910>),
 ('cherry', <pyspark.resultiterable.ResultIterable at 0x7ff71a60e190>)]

In [264]:
reduced_rdd = kv_rdd.reduceByKey(lambda x,y : x+y)
reduced_rdd.collect()

[('apple', 1), ('banana', 2), ('cherry', 3)]

In [266]:

# 셔플링이 필요한 변환 (Wide Transformation)
distinct_rdd = sc.parallelize([1, 1, 2, 2, 3, 3]).distinct()
grouped_rdd = kv_rdd.groupByKey()
reduced_rdd = kv_rdd.reduceByKey(lambda x, y: x + y)

In [268]:
# 두 RDD 조인하기
rdd1 = sc.parallelize([("apple", 2), ("banana", 1)])
rdd2 = sc.parallelize([("apple", "fruit"), ("banana", "fruit"), ("carrot", "vegetable")])

# 두 RDD 조인
joined_rdd = rdd1.join(rdd2)
print(joined_rdd.collect())
# 출력 결과: [('apple', (2, 'fruit')), ('banana', (1, 'fruit'))]

print("Distinct RDD:", distinct_rdd.collect())  # [1, 2, 3]
print("Joined RDD:", joined_rdd.collect())  # [("apple", (2, "fruit")), ("banana", (1, "fruit"))]

[('apple', (2, 'fruit')), ('banana', (1, 'fruit'))]
Distinct RDD: [2, 1, 3]
Joined RDD: [('apple', (2, 'fruit')), ('banana', (1, 'fruit'))]


In [269]:
# 왼쪽 외부 조인 (leftOuterJoin)
left_joined = rdd1.leftOuterJoin(rdd2)
print(left_joined.collect())
# 출력 결과: [('apple', (2, 'fruit')), ('banana', (1, 'fruit'))]

# 오른쪽 외부 조인 (rightOuterJoin)
right_joined = rdd1.rightOuterJoin(rdd2)
print(right_joined.collect())
# 출력 결과: [('apple', (2, 'fruit')), ('banana', (1, 'fruit')), ('carrot', (None, 'vegetable'))]

# subtractByKey 연산
subtract_result = rdd1.subtractByKey(sc.parallelize([("apple", "any")]))
print(subtract_result.collect())
# 출력 결과: [('banana', 1)]

[('apple', (2, 'fruit')), ('banana', (1, 'fruit'))]
[('apple', (2, 'fruit')), ('banana', (1, 'fruit')), ('carrot', (None, 'vegetable'))]
[('banana', 1)]


In [272]:
# reduce 연산 (결합성과 교환성을 가진 작업)
sum_result = numbers.reduce(lambda x, y: x + y)
product_result = numbers.reduce(lambda x, y: x * y)


print("Sum:", sum_result)  # 15
print("Product:", product_result)  # 120

Sum: 15
Product: 120


In [273]:
from operator import add

In [274]:
sample_rdd = sc.parallelize([1, 2, 3, 4, 5])
sample_rdd

ParallelCollectionRDD[289] at readRDDFromFile at PythonRDD.scala:289

In [276]:
#RDD에 있는 숫자의 합계를 계산:
sample_rdd.reduce(add)

15

In [277]:
sample_rdd = sc.parallelize([1, 2, 3, 4])
sample_rdd.reduce(lambda x, y : (x * 2)+ y)

18

In [279]:
sc.parallelize([1, 2, 3, 4], 2).reduce(lambda x, y : (x * 2) + y)

18

In [280]:
sc.parallelize([1, 2, 3, 4], 3).reduce(lambda x, y : (x * 2) + y)

18

In [281]:
sc.parallelize([1, 2, 3, 4], 1).reduce(lambda x, y : (x * 2) + y)

26

In [282]:
# 파티션을 3개로 지정

In [283]:
rdd = sc.parallelize([2, 3, 4], 4)

#숫자의 곱을 계산해서 합한다
print(rdd.reduce(lambda x, y : (x * y))) # 2 x 3 -> x 4
print(rdd.fold(1, lambda x , y : (x * y))) # 1 x 2 x 3 x 4

24
24


In [284]:
rdd.fold(2, lambda x , y : (x * y)) 
# 초기값 2 지정 :: 2x2 , 2x3, 2x4, "" > 4 , 6, 8, "" 
# 다시 초기값 2 지정 :: 4x2*6, 48x2*8, "" > 768

768

In [285]:
print(rdd.reduce(lambda x, y : x + y)) 
print(rdd.fold(1, lambda x, y : x + y)) 

9
14


In [286]:
# 대규모 숫자 데이터셋 생성
numbers_rdd = sc.parallelize(range(1, 1000001))

# fold를 사용하여 총합계 계산
total_sum = numbers_rdd.fold(0, lambda x, y: x + y)

print(f"Total sum: {total_sum}")

Total sum: 500000500000


In [287]:
# 직원 급여 데이터
salaries_rdd = sc.parallelize([50000, 60000, 55000, 75000, 65000, 80000])

# fold를 사용하여 최대 급여 찾기
max_salary = salaries_rdd.fold(0, lambda x, y: max(x, y))

print(f"Maximum salary: {max_salary}")

Maximum salary: 80000


In [288]:
# 부서별 직원 급여 데이터
dept_salaries = [("IT", 50000), ("HR", 45000), ("IT", 60000), 
                 ("Finance", 55000), ("HR", 50000), ("IT", 65000)]
dept_rdd = sc.parallelize(dept_salaries)

# foldByKey를 사용하여 부서별 최대 급여 찾기
max_salary_by_dept = dept_rdd.foldByKey(0, lambda x, y: max(x, y))

print("Maximum salary by department:")
for dept, max_salary in max_salary_by_dept.collect():
    print(f"{dept}: {max_salary}")

Maximum salary by department:
IT: 65000
HR: 50000
Finance: 55000


In [296]:
rdd = sc.parallelize([1, 1, 2, 3, 5, 8])
result = rdd.groupBy(lambda x : x % 2).collect()

sorted([(x, sorted(y)) for (x, y) in result])

[(0, [2, 8]), (1, [1, 1, 3, 5])]

In [297]:
[(x, sorted(y)) for (x, y) in result]

[(0, [2, 8]), (1, [1, 1, 3, 5])]

In [298]:
#판매 데이터 분석 (카테고리별 총 매출 계산)
sales_data = [
    ("Electronics", 1000),
    ("Clothing", 500),
    ("Electronics", 1500),
    ("Books", 300),
    ("Clothing", 750),
    ("Books", 200)
]

In [299]:
# RDD 생성
sales_rdd = sc.parallelize(sales_data)

# 카테고리별 총 매출 계산
category_sales = sales_rdd.groupBy(lambda x: x[0]) \
    .mapValues(lambda values: sum(item[1] for item in values)) \
    .collect()

# 결과 출력
for category, total_sales in category_sales:
    print(f"{category}: ${total_sales}")

Clothing: $1250
Electronics: $2500
Books: $500


In [295]:
#로그 데이터 처리 (IP 주소별 접속 횟수 분석)
# SparkContext 생성 가정 (sc)
log_data = [
    ("192.168.1.1", "login"),
    ("192.168.1.2", "purchase"),
    ("192.168.1.1", "view"),
    ("192.168.1.3", "login"),
    ("192.168.1.2", "login"),
    ("192.168.1.1", "purchase")
]

# RDD 생성
log_rdd = sc.parallelize(log_data)

In [309]:
ip_count_rdd = log_rdd.map(lambda log: (log[0], 1))

# reduceByKey를 사용하여 같은 IP를 가진 모든 값을 합산합니다.
# 이 연산은 데이터를 재분배(셔플)하므로 Wide 트랜스포메이션입니다.
total_counts = ip_count_rdd.reduceByKey(lambda a, b: a + b)

# 결과를 화면에 출력합니다.
# collect()는 RDD의 모든 요소를 드라이버로 가져오는 액션(Action)입니다.
print("--- IP 주소별 행위 횟수 ---")
for ip, count in total_counts.collect():
    print(f"IP: {ip}, 횟수: {count}")

--- IP 주소별 행위 횟수 ---
IP: 192.168.1.1, 횟수: 3
IP: 192.168.1.2, 횟수: 2
IP: 192.168.1.3, 횟수: 1


In [346]:
log_rdd = sc.parallelize(log_data)

login_rdd = log_rdd.filter(lambda log: log[1] == "login")
view_rdd = log_rdd.filter(lambda log: log[1] == "view")
purchase_rdd = log_rdd.filter(lambda log: log[1] == "purchase")

#################################################################
ip_login_rdd = login_rdd.map(lambda log: (log[0], 1))
ip_view_rdd = view_rdd.map(lambda log: (log[0], 1))
ip_purchase_rdd = purchase_rdd.map(lambda log: (log[0], 1))
######################################################################


# reduceByKey를 사용하여 같은 IP를 가진 모든 값을 합산- ip value
total_login_counts = ip_login_rdd.reduceByKey(lambda a, b: a + b)
total_view_counts = ip_view_rdd.reduceByKey(lambda a, b: a + b)
total_purchase_counts = ip_purchase_rdd.reduceByKey(lambda a, b: a + b)

count_=0
#각 행위의 값을 출력
print("--- IP 주소별 login 횟수 ---")
for ip, count in total_login_counts.collect():
    print(f"IP {ip} login 횟수: {count}")
    count_=count_+1

print(f"총 login 횟수:{count_}")
count_=0
for ip, count in total_view_counts.collect():
    print(f"IP {ip} view 횟수: {count}")
    count_=count_+1
print(f"총 view 횟수:{count_}")
count_=0
for ip, count in total_purchase_counts.collect():
    print(f"IP {ip} purchase 횟수: {count}")
    count_=count_+1
print(f"총 purchase 횟수:{count_}")



--- IP 주소별 login 횟수 ---
IP 192.168.1.1 login 횟수: 1
IP 192.168.1.3 login 횟수: 1
IP 192.168.1.2 login 횟수: 1
총 login 횟수:3
IP 192.168.1.1 view 횟수: 1
총 view 횟수:1
IP 192.168.1.2 purchase 횟수: 1
IP 192.168.1.1 purchase 횟수: 1
총 purchase 횟수:2


In [347]:
from pyspark import SparkContext

sc = SparkContext.getOrCreate()
numbers = sc.parallelize([1, 5, 3, 9, 2, 8, 4, 7, 6], numSlices=3)
numbers.collect()

[1, 5, 3, 9, 2, 8, 4, 7, 6]

In [348]:
numbers.glom().collect()

[[1, 5, 3], [9, 2, 8], [4, 7, 6]]

In [350]:
numbers = sc.parallelize([1, 5, 3, 9, 2, 8, 4, 7, 6])

In [355]:
# 평균 계산하기
sum_count = numbers.aggregate(
    (0, 0),  # 초기값 (합계, 개수)
    lambda acc, value: (acc[0] + value, acc[1] + 1),  # 각 파티션 내 연산
    lambda acc1, acc2: (acc1[0] + acc2[0], acc1[1] + acc2[1])  # 파티션 간 연산
)
sum_count

(45, 9)

In [356]:
average = sum_count[0] / sum_count[1]
print("Average:", average)  # 5.0

Average: 5.0


In [357]:
# agg 연습
data = ["hello", "world", "spark"]
rdd = sc.parallelize(data)

def seq_op(acc, value):
    text, idx = acc
    return (f"{text} {idx}:{value}".strip(), idx + 1)

def comb_op(acc1, acc2):
    # 줄 번호 이어붙이기: 앞쪽 text 유지 + 뒤쪽 text 뒤에 붙이기
    text1, idx1 = acc1
    text2, idx2 = acc2
    # 줄 번호 충돌 피하려면 idx 조정 필요 (복잡하므로 단순히 붙임)
    return (f"{text1} {text2}".strip(), idx1 + idx2)

In [358]:
zero = ("", 1)

result = rdd.aggregate(zero, seq_op, comb_op)[0]
print(result)

1:hello 1:world 2:spark


In [359]:
rdd = sc.parallelize(["spark", "hadoop", "ai", "python", "sql"])
result = rdd.aggregate(zero, seq_op, comb_op)[0]
print(result)

1:spark 2:hadoop 1:ai 2:python 3:sql


In [360]:
# 길이가 5 이상인 문자열 개수 세기
rdd = sc.parallelize(["spark", "hadoop", "ai", "python", "sql"])

result = rdd.aggregate(
    0,
    lambda acc, value: acc + (1 if len(value) >= 5 else 0),
    lambda acc1, acc2: acc1 + acc2
)

print("길이 5 이상인 문자열 개수:", result)


길이 5 이상인 문자열 개수: 3


In [367]:
spark.stop()

In [368]:
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster("local").setAppName("spark_sql_basic2")
sc   = SparkContext(conf=conf)

In [369]:
movies_rdd = sc.parallelize([
    (1, ("어벤져스", "마블")),
    (2, ("슈퍼맨", "DC")),
    (3, ("배트맨", "DC")),
    (4, ("겨울왕국", "디즈니")),
    (5, ("아이언맨", "마블"))
])


attendances_rdd = sc.parallelize([
    (1, (13934592, "KR")),
    (2, (2182227,"KR")),
    (3, (4226242, "KR")),
    (4, (10303058, "KR")),
    (5, (4300365, "KR"))
])

In [380]:
sc.stop()

In [386]:
spark.stop()

In [372]:
# CASE1. join 먼저, filter 나중에
movie_att = movies_rdd.join(attendances_rdd)
movie_att.take(5)

[(2, (('슈퍼맨', 'DC'), (2182227, 'KR'))),
 (4, (('겨울왕국', '디즈니'), (10303058, 'KR'))),
 (1, (('어벤져스', '마블'), (13934592, 'KR'))),
 (3, (('배트맨', 'DC'), (4226242, 'KR'))),
 (5, (('아이언맨', '마블'), (4300365, 'KR')))]

In [373]:
movie_att.filter(
    lambda x : x[1][0][1] == "마블" and x[1][1][0] > 5000000
).collect()

[(1, (('어벤져스', '마블'), (13934592, 'KR')))]

In [371]:
# CASE 2. filter 먼저, join 나중에
filtered_movies = movies_rdd.filter(lambda x : x[1][1] == '마블')
filtered_att = attendances_rdd.filter(lambda x : x[1][0] > 5000000)

filtered_movies.join(filtered_att).collect()

[(1, (('어벤져스', '마블'), (13934592, 'KR')))]

In [388]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("spark-sql").getOrCreate()

In [387]:
# 컬럼 추가
movies = [
    (1, "어벤져스", "마블", 2012, 4, 26),
    (2, "슈퍼맨", "DC", 2013, 6, 13),
    (3, "배트맨", "DC", 2008, 8, 6),
    (4, "겨울왕국", "디즈니", 2014, 1, 16),
    (5, "아이언맨", "마블", 2008, 4, 30)
]


In [389]:
#스키마를 알아야 한다.
movie_schema = ["id", "name", "company", "year", "month", "day"]

In [390]:
df = spark.createDataFrame(data=movies, schema=movie_schema)

In [392]:
df.select("name").show()

+--------+
|    name|
+--------+
|어벤져스|
|  슈퍼맨|
|  배트맨|
|겨울왕국|
|아이언맨|
+--------+



In [393]:
df.filter(df.year >= 2010).show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  1|어벤져스|   마블|2012|    4| 26|
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  4|겨울왕국| 디즈니|2014|    1| 16|
+---+--------+-------+----+-----+---+



In [397]:
df.createOrReplaceTempView("movies")

In [398]:
# 영화 이름만 가져오기

query = """

SELECT name
  FROM movies

"""
spark.sql(query).show()

+--------+
|    name|
+--------+
|어벤져스|
|  슈퍼맨|
|  배트맨|
|겨울왕국|
|아이언맨|
+--------+



In [400]:
df.columns

['id', 'name', 'company', 'year', 'month', 'day']

In [401]:
df.dtypes

[('id', 'bigint'),
 ('name', 'string'),
 ('company', 'string'),
 ('year', 'bigint'),
 ('month', 'bigint'),
 ('day', 'bigint')]

In [410]:
df.select("name").show()

+--------+
|    name|
+--------+
|어벤져스|
|  슈퍼맨|
|  배트맨|
|겨울왕국|
|아이언맨|
+--------+



In [403]:
# 2010년 이후에 개봉한 영화를 조회

query = """

SELECT name
  FROM movies where year>2010

"""
spark.sql(query).show()

+--------+
|    name|
+--------+
|어벤져스|
|  슈퍼맨|
|겨울왕국|
+--------+



In [404]:
# 2012년도 이전에 개봉한 영화의 이름과 회사를 출력
query = """
SELECT name
  FROM movies where year<2012

"""
spark.sql(query).show()

+--------+
|    name|
+--------+
|  배트맨|
|아이언맨|
+--------+



In [411]:
# like 문자열 데이터에서 특정 단어나 문장을 포함한 데이터를 찾을 때
# % 기호를 사용해서 문장이 매칭되는지 확인 가능!
# 제목이 ~~맨으로 끝나는 데이터의 모든 정보를 조회
query = """

SELECT name
  FROM movies where name like "_%맨"

"""
spark.sql(query).show()

+--------+
|    name|
+--------+
|  슈퍼맨|
|  배트맨|
|아이언맨|
+--------+



In [412]:
df.filter(df.year >2010).show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  1|어벤져스|   마블|2012|    4| 26|
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  4|겨울왕국| 디즈니|2014|    1| 16|
+---+--------+-------+----+-----+---+



In [413]:
df.select("day").show()

+---+
|day|
+---+
| 26|
| 13|
|  6|
| 16|
| 30|
+---+



In [420]:
df.select("day", "month", "year" ).show()

+---+-----+----+
|day|month|year|
+---+-----+----+
| 26|    4|2012|
| 13|    6|2013|
|  6|    8|2008|
| 16|    1|2014|
| 30|    4|2008|
+---+-----+----+



In [425]:
df.filter(df.year >2013).show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  4|겨울왕국| 디즈니|2014|    1| 16|
+---+--------+-------+----+-----+---+



In [438]:
df.filter((df.company =="마블") | (df.company =="DC") ).show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  1|어벤져스|   마블|2012|    4| 26|
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  3|  배트맨|     DC|2008|    8|  6|
|  5|아이언맨|   마블|2008|    4| 30|
+---+--------+-------+----+-----+---+



In [440]:
df.select(df.name, (df.month>4) & (df.month<8) ).show()

+--------+-----------------------------+
|    name|((month > 4) AND (month < 8))|
+--------+-----------------------------+
|어벤져스|                        false|
|  슈퍼맨|                         true|
|  배트맨|                        false|
|겨울왕국|                        false|
|아이언맨|                        false|
+--------+-----------------------------+



In [442]:
df.filter((df.month >= 4) & (df.month <= 8)).show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  1|어벤져스|   마블|2012|    4| 26|
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  3|  배트맨|     DC|2008|    8|  6|
|  5|아이언맨|   마블|2008|    4| 30|
+---+--------+-------+----+-----+---+

